
#  Game Analysis

##  Heroes of Pymoli

* Of the 576 players engaging in in-game purchases, the vast majority at 84% are male. There also exists a smaller, but notable, proportion of female players at 14%.

* The percentage of purchases follows a similar split down gender lines of 83.6% male and 14.5% female, though the average female ends up spending just under 10% more than the average male and she spends about 6% more per transaction. 

* The peak age demographic falls between ages 20-24 (44.8%) surrounded by the next best at ages 15-19 (18.60%) and 25-29 (13.4%).

* Looking at the top 10% of items in terms of popularity and profitability, the most popular items are not necessarily the most profitable given the relatively small number of purchases (a 9 way tie for 3rd place with 8 purchases and a difference of 1 purchase between 2nd, 3rd and 4th).  

* Most of the top 10% of profitable items have a price of at least $3.22.

-----
### Setup

In [1]:
#  Dependencies
import pandas as pd
import numpy as np


In [2]:
#  Get source data for analysis
sourcefile = "Resources/purchase_data.csv"
raw_csv = pd.read_csv(sourcefile)

#  Preview source data
raw_csv.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


-----
### Player Count

In [3]:
#  Identify the number of purchases made by each player who engaged in a transaction
player_purchases = raw_csv["SN"].value_counts()

#  Identify total number of players who engaged in a transaction
player_count = pd.DataFrame({"Total Players" : [len(player_purchases)]})

#  Display results
player_count

,Total Players
0,576


-----
### Summary of Purchase Data

In [4]:
#  ID which types of items were bought
items_bought = raw_csv["Item ID"].value_counts()     # NOTE: there are 4 items that have multiple ID's with the same name!

#  ID how many different items were bought
items_count = len(items_bought)

#  Average amount spent on a purchase
avg_price = raw_csv["Price"].mean()

#  Get the number of total transactions
trans_count = len(raw_csv["Purchase ID"].unique())

#  Determine the total amount 
total_value = raw_csv["Price"].sum()

#  Summarize findings
purchase_summary = pd.DataFrame({"Number of Unique Items":[items_count], 
                                 "Average Price":["${:,.2f} ".format(avg_price)],
                                 "Number of Purchases":[trans_count],
                                 "Total Revenue":["${:,.2f} ".format(total_value)]
                                })
#  Display results
purchase_summary

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


-----
### Gender Demographics Data

In [5]:
#  Get gender breakdowns on purchases
gender_grp = raw_csv.groupby(["Gender"])

#  Get unique ID's within the gender groups
gender_SNs = gender_grp["SN"].unique()

#  Figure out the counts of purchasers for each gender
gender_count = [len(i) for i in gender_SNs]

#  Gather results into a new data frame
gender_data = pd.DataFrame(gender_SNs)
gender_data["Total Count"] = gender_count

#  Add in percentage information
gender_data["Percentage in Decimal"] = gender_data["Total Count"] / gender_data["Total Count"].sum()

#  Formated version of the percentage information
gender_data["Percentage of Players"] = ["{:,.2f}%".format(i * 100) for i in gender_data["Percentage in Decimal"]]

#  Summarize findings
gender_dem_summ = gender_data[["Total Count", "Percentage of Players"]].sort_values(["Total Count"], ascending=False)

#  Display results
gender_dem_summ

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


-----
### Gender Purchase Analysis 

In [6]:
#  Number of purchases by gender allowing for multiple purchases from the same ID
gender_data["Purchase Count"] = gender_grp["Purchase ID"].count()

#  Total spend of each gender group
gender_data["Total Spend"] = gender_grp["Price"].sum()

#  Average purchase price by gender
gender_data["Average Price"] = gender_data["Total Spend"] / gender_data["Purchase Count"]

#  Average amount spent by gender
gender_data["Average Spend"] = gender_data["Total Spend"] / gender_data["Total Count"] 

#  Format numeric results for readability
gender_data["Average Purchase Price"] = ["${:,.2f} ".format(i) for i in gender_data["Average Price"]]
gender_data["Total Purchase Value"] = ["${:,.2f} ".format(i) for i in gender_data["Total Spend"]]
gender_data["Avg Total Purchase per Person"] = ["${:,.2f} ".format(i) for i in gender_data["Average Spend"]]

#  Summarize findings
gender_purch_summ = gender_data[["Purchase Count", 
                            "Average Purchase Price", 
                            "Total Purchase Value", 
                            "Avg Total Purchase per Person"
                           ]].sort_values(["Purchase Count"], ascending=False)

#  Display results
gender_purch_summ

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Male,652,$3.02,"$1,967.64",$4.07
Female,113,$3.20,$361.94,$4.47
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


-----
### Age Demographics Data

In [7]:
#  Bins for age groups
#   "Round" number age groups
round_bin = [0, 9.9167, 14.9167, 19.9167, 24.9167, 29.9167, 34.9167, 39.9167, 120]
round_lbl = ["Under 10", "10 - 14", "15 - 19", "20 - 24", "25 - 29",  "30 - 34", "35 - 39", "40+"]

#   Age grouped on generation
cur_yr = 2018
gen_bin = [0,
           cur_yr - 2013, # Gen Alpha
           cur_yr - 1995, # Gen Z
           cur_yr - 1985, # Millenials
           cur_yr - 1980, # Gen Y
           cur_yr - 1965, # Gen X
           cur_yr - 1946, # Baby Boomers
           cur_yr - 1925, # Silent Gen
           cur_yr - 1910, # Greatest Gen
           cur_yr - 1901  # Interbellum
           ]
gen_lbl = ["Gen Alpha", "Gen Z", "Millenials", "Gen Y", "Gen X", 
           "Baby Boomers", "Silent Gen", "Greatest Gen", "Interbellum"]

#   Ages around school/work milestones
mile_bin = [0, 
              11, # Grade school
              13, # Middle school
              17, # High school
              21, # Undergrad
              25, # Post-grad
              30, # Young Adult
              55, # Adult
              65, # AARP eligible -_-
              120 # Senior
             ]
mile_lbl = ["Grade School", "Middle School", "High School", 
            "Undergrads", "Post-Grads", "Young Adults", 
            "Adults", "AARP Eligible", "Seniors"]

In [8]:
#  Add age range to a copy of the data based on desired bins and labels
#   Edit here for alternative groupings
edit_csv = raw_csv
edit_csv["Age Range"] = pd.cut(edit_csv["Age"], round_bin, labels=round_lbl)

#  Group the data by the new age range category
age_demo = edit_csv.groupby("Age Range")

In [9]:
#  Count the number of unique user handles in each age range into a data frame
age_data = pd.DataFrame(age_demo["SN"].unique())
age_count = [len(i) for i in age_data["SN"]]
age_data["Total Count"] = age_count

#  Add in percentage measures
age_data["Pct Calc"] = age_data["Total Count"] / age_data["Total Count"].sum()

#  Format numbers
age_data["Percentage of Players"] = ["{:,.2f}%".format(i * 100) for i in age_data["Pct Calc"]]

#  Extract Summary Information
age_demo_summ = age_data[["Total Count", "Percentage of Players"]]

#  Display results
age_demo_summ

,Total Count,Percentage of Players
Age Range,,
Under 10,17,2.95%
10 - 14,22,3.82%
15 - 19,107,18.58%
20 - 24,258,44.79%
25 - 29,77,13.37%
30 - 34,52,9.03%
35 - 39,31,5.38%
40+,12,2.08%


-----
### Age Purchase Analysis

In [10]:
#  Count total purchases by age range
age_data["Purchase Count"] = age_demo["Purchase ID"].count()

#  Determine total spent by age
age_data["Tot Spend"] = age_demo["Price"].sum()

#  Determine average transaction price
age_data["Avg Trans Spend"] = age_data["Tot Spend"] / age_data["Purchase Count"]

#  Determine the average spend per person
age_data["Avg Pers Spend"] = age_data["Tot Spend"] / age_data["Total Count"]

#  Format numbers for currency and percentage
age_data["Average Purchase Price"] = ["${:,.2f} ".format(i) for i in age_data["Avg Trans Spend"]]
age_data["Total Purchase Value"] = ["${:,.2f} ".format(i) for i in age_data["Tot Spend"]]
age_data["Avg Total Purchase per Person"] = ["${:,.2f} ".format(i) for i in age_data["Avg Pers Spend"]]

#  Extract Summary
age_purch_summ = age_data[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"]]

#  Display results
age_purch_summ

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Range,,,,
Under 10,23,$3.35,$77.13,$4.54
10 - 14,28,$2.96,$82.78,$3.76
15 - 19,136,$3.04,$412.89,$3.86
20 - 24,365,$3.05,"$1,114.06",$4.32
25 - 29,101,$2.90,$293.00,$3.81
30 - 34,73,$2.93,$214.00,$4.12
35 - 39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


-----
### Top Spenders

#### Specify the number of desired results here:

In [11]:
#  Desired number of results
results = 5

In [12]:
#  Group up transactions by user handle
player_grp = raw_csv.groupby("SN")

#  Get sums of the prices of their transactions
player_data = pd.DataFrame(player_grp["Price"].sum())#.sort_values(["Price"], ascending=False)

#  Get the total number of transactions per handle
player_data["Purchase Count"] = player_grp["Price"].count()

#  Calculate the average purchase per player
player_data["Avg Spend"] = player_data["Price"] / player_data["Purchase Count"]

#  Sort by total spent amount
player_data = player_data.sort_values(["Price"], ascending=False)

#  Format numbers for currency
player_data["Total Purchase Value"] = ["${:,.2f} ".format(i) for i in player_data["Price"]]
player_data["Average Purchase Price"] = ["${:,.2f} ".format(i) for i in player_data["Avg Spend"]]

#  Extract desired data
top_spenders = player_data[["Total Purchase Value", "Purchase Count", "Average Purchase Price"]].head(results) 

#  Display results
top_spenders

,Total Purchase Value,Purchase Count,Average Purchase Price
SN,,,
Lisosia93,$18.96,5,$3.79
Idastidru52,$15.45,4,$3.86
Chamjask73,$13.83,3,$4.61
Iral74,$13.62,4,$3.40
Iskadarya95,$13.10,3,$4.37


-----
### Most Popular Items
-----
#### By Item ID

In [13]:
#  Extract Item ID and matching info
item_grp = raw_csv[["Item ID", "Item Name", "Price"]].groupby(["Item ID", "Item Name"])

#  Find total amount spent on Item ID
item_data = pd.DataFrame(item_grp["Price"].sum())  # Not going to rename Price to Total pre-formatting
#item_data["ID Names"] = item_grp["Item Name"].unique()

#  Get purchase counts per ID
item_data["Purchase Count"] = item_grp["Price"].count()

#  Get the average price per item
item_data["Avg Spend"] = item_data["Price"] / item_data["Purchase Count"]

#  Compare average price to price list to check if prices have ever fluctuated
     #item_data["Price List"] = item_grp["Price"].unique()
     #item_data["Price Count"] = [len(i) for i in item_data["Price List"]]
     #dupe_check = item_data.sort_values(["Price Count"], ascending=False)
     #dupe_check
        
#  Sort by total purchases on ID
item_data = item_data.sort_values("Purchase Count", ascending=False)

#  Format Currencies
item_data["Total Purchase Value"] = ["${:,.2f} ".format(i) for i in item_data["Price"]]
item_data["Item Price"] = ["${:,.2f} ".format(i) for i in item_data["Avg Spend"]]

#  Summarize findings
item_summ = item_data[["Purchase Count", "Total Purchase Value", "Item Price"]].head(results)

#  Display results
item_summ

,,Purchase Count,Total Purchase Value,Item Price
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76,$4.23
145,Fiery Glass Crusader,9,$41.22,$4.58
108,"Extraction, Quickblade Of Trembling Hands",9,$31.77,$3.53
82,Nirvana,9,$44.10,$4.90
19,"Pursuit, Cudgel of Necromancy",8,$8.16,$1.02


#### By Item Name

In [14]:
#  Extract Item ID and matching info
alt_item_grp = raw_csv[["Item ID", "Item Name", "Price"]].groupby(["Item Name"])

#  Find total amount spent on Item ID
alt_item_data = pd.DataFrame(alt_item_grp["Price"].sum())  # Not going to rename Price to Total pre-formatting
alt_item_data["Item ID's"] = alt_item_grp["Item ID"].unique()
alt_item_data["ID Count"] = [len(i) for i in alt_item_data["Item ID's"]]


* There are 4 items with the same name, but multiple Item ID's.  This could be due to seasonal re-releases of the item or just a coding error.  Un-comment cell below to see the aforementioned items.

In [15]:
#  Check to see which items have duplicate ID's
#dupe_check = alt_item_data.sort_values(["ID Count"], ascending=False)
#dupe_check.head() 

In [16]:
#  Get purchase counts per ID
alt_item_data["Purchase Count"] = alt_item_grp["Price"].count()

#  Get the average price per item
alt_item_data["Avg Spend"] = alt_item_data["Price"] / alt_item_data["Purchase Count"]

        
#  Sort by total purchases on Name
alt_item_data = alt_item_data.sort_values("Purchase Count", ascending=False)

#  Format Currencies
alt_item_data["Total Purchase Value"] = ["${:,.2f} ".format(i) for i in alt_item_data["Price"]]
alt_item_data["Item Price"] = ["${:,.2f} ".format(i) for i in alt_item_data["Avg Spend"]]

#  Summarize findings
alt_item_summ = alt_item_data[["Item ID's", "Purchase Count", "Total Purchase Value", "Item Price"]].head(results)

#  Display results
alt_item_summ

,Item ID's,Purchase Count,Total Purchase Value,Item Price
Item Name,,,,
Final Critic,"[92, 101]",13,$59.99,$4.61
"Oathbreaker, Last Hope of the Breaking Storm",[178],12,$50.76,$4.23
Persuasion,"[141, 132]",9,$28.99,$3.22
Nirvana,[82],9,$44.10,$4.90
"Extraction, Quickblade Of Trembling Hands",[108],9,$31.77,$3.53


-----
### Most Profitable Items
-----
#### By Item ID

In [17]:
#  Sort data by total spent on ID
item_data = item_data.sort_values("Price", ascending=False)

#  Summarize findings
profit_summ = item_data[["Total Purchase Value", "Purchase Count", "Item Price"]]

#  Display results
profit_summ.head(results)

,,Total Purchase Value,Purchase Count,Item Price
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",$50.76,12,$4.23
82,Nirvana,$44.10,9,$4.90
145,Fiery Glass Crusader,$41.22,9,$4.58
92,Final Critic,$39.04,8,$4.88
103,Singed Scalpel,$34.80,8,$4.35


#### By Item Name

In [18]:
#  Sort data by total spent on Name
alt_item_data = alt_item_data.sort_values("Price", ascending=False)

#  Summarize findings
alt_prof_summ = alt_item_data[["Item ID's", "Total Purchase Value", "Purchase Count", "Item Price"]]

#  Display Reults
alt_prof_summ.head(results)

,Item ID's,Total Purchase Value,Purchase Count,Item Price
Item Name,,,,
Final Critic,"[92, 101]",$59.99,13,$4.61
"Oathbreaker, Last Hope of the Breaking Storm",[178],$50.76,12,$4.23
Nirvana,[82],$44.10,9,$4.90
Fiery Glass Crusader,[145],$41.22,9,$4.58
Singed Scalpel,[103],$34.80,8,$4.35
